**The purpose of this project is to analyse the location a taxi driver should work to make the most profit.**

I will analyse, if possible, all yellow, green, and FHV taxis, compare them and draw conclusions with why they might be the same or different. Obviously there will be obstacles, but I'll just sort of wing it and solve every roadblocks or problems that might occur throughout the project.

(I hope I can stick with this idea and I don't have to change topics in the middle of doing it)

In [1]:
# this is to import my py script which is located in a different folder
import sys
sys.path.insert(0, '../scripts')

from readparquet import read_parquet
sdf = read_parquet('../data/raw/fhvhv_tripdata_2022-01.parquet')
# convert to pandas dataframe for visualisation
df = sdf.sample(0.001, seed=0).toPandas()

22/07/29 16:04:33 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 192.168.207.1 instead (on interface eth1)
22/07/29 16:04:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/29 16:04:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.info()
df.sample(n=5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     14731 non-null  object        
 1   dispatching_base_num  14731 non-null  object        
 2   originating_base_num  10793 non-null  object        
 3   request_datetime      14731 non-null  datetime64[ns]
 4   on_scene_datetime     10795 non-null  datetime64[ns]
 5   pickup_datetime       14731 non-null  datetime64[ns]
 6   dropoff_datetime      14731 non-null  datetime64[ns]
 7   PULocationID          14731 non-null  int64         
 8   DOLocationID          14731 non-null  int64         
 9   trip_miles            14731 non-null  float64       
 10  trip_time             14731 non-null  int64         
 11  base_passenger_fare   14731 non-null  float64       
 12  tolls                 14731 non-null  float64       
 13  bcf             

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
12633,HV0003,B03404,B03404,2022-01-28 04:28:04,2022-01-28 04:29:55,2022-01-28 04:30:04,2022-01-28 04:36:12,98,9,1.470,...,0.82,0.00,0.0,0.00,6.04,N,N,,N,N
549,HV0003,B03404,B03404,2022-01-02 17:11:49,2022-01-02 17:16:55,2022-01-02 17:18:33,2022-01-02 17:35:41,145,138,7.770,...,2.11,0.00,2.5,5.31,18.17,N,N,,N,N
716,HV0003,B03404,B03404,2022-01-03 03:05:46,2022-01-03 03:07:20,2022-01-03 03:08:34,2022-01-03 03:16:37,14,67,1.330,...,0.67,0.00,0.0,0.00,7.76,N,N,,N,N
1917,HV0005,B03406,None,2022-01-06 02:34:08,NaT,2022-01-06 02:35:27,2022-01-06 02:50:44,162,145,3.045,...,1.86,2.75,0.0,3.94,11.06,N,N,N,N,N
11659,HV0003,B03404,B03404,2022-01-26 04:30:18,2022-01-26 04:32:16,2022-01-26 04:33:15,2022-01-26 04:58:24,144,265,4.360,...,0.00,0.00,0.0,0.00,22.51,N,N,,N,N


Hmmmm, it seems like there are too many features necessary for data analysis. Let's remove the ones I am certain are completely useless.

In [3]:
from pyspark.sql.functions import col
sdf_s = sdf.select(col("pickup_datetime"), col("dropoff_datetime"), col("PULocationID"), col("DOLocationID"), \
    col("trip_miles"), col("base_passenger_fare"), col("driver_pay"), col("tips"))
df_s = sdf_s.sample(0.001, seed=0).toPandas()
df_s.sample(n=5)

,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,base_passenger_fare,driver_pay,tips
12169,2022-01-27 05:48:36,2022-01-27 05:57:55,260,157,2.628,10.72,7.60,0.0
3323,2022-01-09 06:29:52,2022-01-09 06:39:41,233,163,1.410,8.23,6.48,0.0
4861,2022-01-12 19:59:00,2022-01-12 20:10:32,17,49,0.870,10.38,12.37,0.0
9266,2022-01-21 17:33:46,2022-01-21 17:52:59,235,126,4.690,22.70,16.89,0.0
5794,2022-01-14 21:45:58,2022-01-14 21:57:34,237,262,1.807,11.62,7.83,0.0


The idea is that, essentially, only the PU location ID will be taken into account for a single taxi ride. 

...

Probably. (Why not DO?)

Anyway, that is the, sort of, overview of what I'll be doing for this project. Now, for the rest of this "data processing" stage, I will read in all the required parquet data, ditch all the useless features, and merge them together such that the data I'll be using for the rest of this project is going to be the "train" data and the "test" data.